In [19]:

!apt updateapt update
!pip install mdbtools-dev
!apt-get install mdbtools
!pip install pandas pyodbc
!pip install pymssql
import warnings
warnings.filterwarnings('ignore')

E: Invalid operation updateapt
ERROR: Could not find a version that satisfies the requirement mdbtools-dev (from versions: none)
ERROR: No matching distribution found for mdbtools-dev
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
mdbtools is already the newest version (1.0.0+dfsg-1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [20]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
import sys
sys.path.append('/content/drive/MyDrive/Dev/py2/')
import numpy as np

import pandas as pd
from google.colab import files
import os
import subprocess
from io import StringIO  # StringIOを正しくインポート
import model.utility.k_analysis_util as k_a_util
from IPython.display import display

from google.colab import drive
import matplotlib.pyplot as plt
import seaborn as sns

import model.utility.k_analysis_util as k_analysis_util

In [22]:
db_path = "/content/drive/MyDrive/Dev/Kjvan/kjdb_prediction_cache_2024.accdb"


In [23]:
horse_id ='2020103049'
cmd = f"select * from HorseResult where rr_r_horse_id='{horse_id}'"
cmd = f"select * from HorseResult"
print(cmd)

select * from HorseResult


In [24]:

k_analysis_util.execute_mdb_sql(db_path, cmd)

"An error occurred: Command '['mdb-sql', '/content/drive/MyDrive/Dev/Kjvan/kjdb_prediction_cache_2024.accdb', '-Q', 'select * from HorseResult']' returned non-zero exit status 1."

In [25]:
import subprocess
import pandas as pd
import io
import tempfile

def execute_mdb_sql(mdb_file_path: str, sql_query: str) -> pd.DataFrame:
    """
    Execute SQL query on Microsoft Access database (.mdb file) using mdb-tools

    Parameters:
    mdb_file_path (str): Path to the .mdb file
    sql_query (str): SQL query to execute

    Returns:
    pandas.DataFrame: Query results as a DataFrame

    Raises:
    subprocess.CalledProcessError: If mdb-sql execution fails
    FileNotFoundError: If mdb file doesn't exist or mdb-tools is not installed
    """
    try:
        # Create temporary file to store SQL query
        with tempfile.NamedTemporaryFile(mode='w', suffix='.sql') as temp_sql:
            # Write SQL query to temp file
            temp_sql.write(sql_query)
            temp_sql.flush()

            # Execute mdb-sql command
            cmd = ['mdb-sql', '-P', '-F', '-d', '\t', mdb_file_path]

            # Run the command and capture output
            process = subprocess.Popen(
                cmd,
                stdin=open(temp_sql.name, 'r'),
                stdout=subprocess.PIPE,
                stderr=subprocess.PIPE,
                text=True
            )

            stdout, stderr = process.communicate()

            if process.returncode != 0:
                raise subprocess.CalledProcessError(
                    process.returncode,
                    cmd,
                    stdout,
                    stderr
                )

            # Convert output to DataFrame
            df = pd.read_csv(
                io.StringIO(stdout),
                sep='\t',
                skiprows=1  # Skip the header row from mdb-sql
            )

            return df

    except FileNotFoundError:
        raise FileNotFoundError(
            "Either the MDB file was not found or mdb-tools is not installed. "
            "Please install mdb-tools using your package manager "
            "(e.g., 'apt-get install mdbtools' on Ubuntu)"
        )



In [26]:
execute_mdb_sql(db_path, cmd)

,rr_r_id,rr_r_race,rr_r_horse_id,rr_r_horse_name,rr_r_horse_no,rr_r_rank,rr_r_waku,rr_r_blinker,rr_r_j_mark,rr_r_age,...,rr_m_blood06,rr_m_blood07,rr_m_blood08,rr_m_blood09,rr_m_blood10,rr_m_blood11,rr_m_blood12,rr_m_blood13,rr_m_blood14,rr_r_breeder_code
0,2024010148,8,2018100037,アザル,7,8,7,0,0,6,...,1220045335,1120001232,1220033049,1120001461,1220031843,1120000941,1220026430,1140002552,1240018104,10334100
1,2024010148,8,2019106695,ピースワンヴェリテ,6,5,6,0,0,5,...,1220043939,1140003690,1240019297,1120001542,1220038935,1120001232,1220033314,1140002939,1240005358,60330200
2,2024010148,8,2019106729,ハクサンメモリー,10,10,8,0,0,5,...,1220058063,1140001909,1240010607,1120001232,1220040289,1120001232,1220038933,1120001164,1220042625,44030600
3,2024010148,8,2020104951,ブルーゴールド,3,1,3,0,0,4,...,1220052017,1120001734,1220046249,1120001752,1220033311,1140002802,1240020382,1140002924,1240024269,8054100
4,2024010148,8,2020109097,ハスノミ,1,9,1,0,0,4,...,1240030074,1140003602,1240025069,1140006801,1240031212,1140004207,1240027977,1140006042,1240030073,60019700
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46593,2024122206,11,2019105748,ブローザホーン,4,0,2,0,0,5,...,1220048459,1140001199,1240018121,1120001743,1220037600,1120001232,1220033055,1120001551,1220031048,23307100
46594,2024122206,11,2020102781,ベラジオオペラ,5,0,3,0,0,4,...,1220055276,1140003006,1220045300,1140002183,1220038829,1140005757,1240026459,1120001232,1220043752,39312600
46595,2024122206,11,2021105143,ダノンデサイル,1,0,1,0,0,3,...,1240030346,1140001199,1240018121,1120001743,1220037600,1140002924,1240024403,1140004403,1240022404,39312600
46596,2024122206,11,2021105436,アーバンシック,3,0,2,0,0,3,...,1220057419,1120001232,1220038933,1140003840,1240024312,1140005757,1240026459,1120001614,1220043928,37312600
